# Example 2 - Solving Poisson's Equation

Completed following the tutorial provided by J. S. Dokken at [jsdokken.com](https://jsdokken.com/dolfinx-tutorial/chapter1/fundamentals.html). This notebook modifies several aspects, such as boundary conditions, to facilitate my own exploration. Distributed under the [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

## Mathematical Background

To summarise the mathematical background, we are solving the Poisson Equation:

\begin{align}
-\nabla^2 u(\mathbf{x}) &= f(\boldsymbol{\mathbf{x}}) && \mathbf{x} \in \Omega\\
u(\mathbf{x}) &= u_D(\mathbf{x})&& \mathbf{x} \in \partial\Omega
\end{align}

Here, $u(\mathbf{x})$ is an unknown quantity, and $f(\mathbf{x})$ the source term (typically zero, and in this case we have Laplace's equation). $\Omega$ is the spatial domain, and $\partial\Omega$ its boundary. This form appears in equations for electrostatics, gravitation, and diffusion processes [(Hunt, 2002)](https://www.damtp.cam.ac.uk/user/reh10/lectures/nst-mmii-chapter2.pdf).

As noted in the tutorial, $\Omega$, $u_D$, and $f$ need to be specified. For the former, a unit square $\Omega =[0,1] \times [0,1]$ is the simplest choice. Let $u_e(x,y) = x^2 + y^2 + 2$. 
\begin{align}
f(x,y) = -\nabla^2 u_e(x,y) \\
\end{align}

\begin{align}
\Delta u_e =  \frac{\partial^2}{\partial x^2}(x^2 + y^2 + 2) + \frac{\partial^2}{\partial y^2}(x^2 + y^2 + 2) \\
\end{align}

And thus,
\begin{align}
f = -4, && u_D(x,y) = x^2 + y^2 + 2
\end{align}

## Mesh and FE Space

First we build a unit square mesh with quad elements, 4 in each direction.

In [1]:
from mpi4py import MPI
from dolfinx import mesh
domain = mesh.create_unit_square(MPI.COMM_SELF, 4, 4, mesh.CellType.quadrilateral)

We then create the finite element space, apply the boundary data (solutions) to all DoFs on the boundary, then create and apply the Dirichlet boundary condition. Dirichlet boundary conditions are fixed - the values of the solution do not change at their respective boundary points.

In [2]:
from dolfinx.fem import functionspace
from dolfinx import fem
import numpy

V = functionspace(domain, ('Lagrange', 1))
uD = fem.Function(V)
uD.interpolate(lambda x: 2 + x[0]**2 + x[1]**2)

tdim = domain.topology.dim # element topological dimension - cells are 2D in this case
fdim = tdim - 1 # facets are therefore edges

domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

## The Variational Problem

In the function space $V$, we now draw two functions. The first, the trial, is the unknown solution we wish to find, and the second, the test, acts as an independent probe to weight the Galerkin residuals, formulating the weak (variational) form. In FEniCSx we draw both from the same function space as $V$ itself does not enforce boundary conditions. Thus we can maintain symmetry, and keep the implementation simpler.

In [3]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [4]:
from dolfinx import default_scalar_type
f = fem.Constant(domain, default_scalar_type(-4)) # As noted by Dokker this expression improves compilation

Now we define the variational problem.

In [5]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

The DOLFINx wrapper for PETSc is used to solve the variational problem.

In [6]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

## Error and Accuracy

We now interpolate the exact solution into a higher-order space, the $P2$-space, to check the solution accuracy.

In [7]:
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 2 + x[0]**2 + x[1]**2)

The scalar form of the $L^2$-norm is computed, as well as the maximum error at any DoF.

In [8]:
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(error_local)

error_max = numpy.max(numpy.abs(uD.x.array-uh.x.array))
print(f"Error_L2 : {error_L2:.2e}")
print(f"Error_max : {error_max:.2e}")

Error_L2 : 2.19e-02
Error_max : 8.88e-16


## Visualisation

The mesh and solution can be visualised using the `pyvista` library to create inline interactive plots.

In [9]:
import pyvista
from dolfinx import plot
 
pyvista.set_jupyter_backend('trame')

domain.topology.create_connectivity(tdim, tdim)
top, cells, geom = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(top, cells, geom)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()
_ = plotter.screenshot("../plots/ex2-mesh.png")

Widget(value='<iframe src="http://localhost:57787/index.html?ui=P_0x175fe9010_0&reconnect=auto" class="pyvista…

### Mesh
![Mesh](../plots/ex2-mesh.png)

In [10]:
u_top, u_cells, u_geom = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_top, u_cells, u_geom)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
u_plotter.show()
_ = u_plotter.screenshot("../plots/ex2-sol.png")

Widget(value='<iframe src="http://localhost:57787/index.html?ui=P_0x137d12fd0_1&reconnect=auto" class="pyvista…

### Solution
![Mesh](../plots/ex2-sol.png)

Finally the mesh can be warped by a scalar, allowing the solution magnitude to be viewed as a height map.

In [11]:
warp_grid = u_grid.warp_by_scalar()
w_plotter = pyvista.Plotter()
w_plotter.add_mesh(warp_grid, show_edges=True, show_scalar_bar=True)
w_plotter.show()
_ = w_plotter.screenshot("../plots/ex2-warp.png")

Widget(value='<iframe src="http://localhost:57787/index.html?ui=P_0x302c07250_2&reconnect=auto" class="pyvista…

### Warped Solution
![Warped Solution](../plots/ex2-warp.png)